In [2]:
using Revise; 
includet("TrajModules.jl")
using .atom_class
using .SystemSetup
using .utils
using .BeamClass
using .CoolTrap
using .IntegrateODE

## Creating the system and light

In [21]:
includet("magneticFields.jl")
using .quadrapole

In [33]:
using BenchmarkTools

In [22]:
BField_Generator = BfieldMag(20.0, 0.0)
Bfield_gen(x,y , z) = Bnorm(BField_Generator, x, y, z)


Bfield_gen (generic function with 1 method)

In [26]:
using Interpolations
function getBFieldInterpolate(Bfield_gen, grid)
    grid = (grid, grid, grid)
    PointGrid = ([i for i in Iterators.product(grid...)])
    B_Field_array = [Bfield_gen(PointGrid[i, j, k]...) for i in 1:size(PointGrid, 1), j in 1:size(PointGrid, 2), k in 1:size(PointGrid,3)]
    B_field_int = interpolate(grid, B_Field_array, Gridded(Linear()))
    B_Field(x, y, z) = B_field_int[x, y, z]
    return B_Field
end
B_Field_int = getBFieldInterpolate(Bfield_gen, -2e-3:0.6e-4:2e-3)


(::var"#B_Field#32"{Interpolations.GriddedInterpolation{Float64, 3, Array{Float64, 3}, Gridded{Linear{Throw{OnGrid}}}, Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}}}) (generic function with 1 method)

In [31]:
function createSystem(grid, B_field)
    atomType = rubidiumAtom
    OurSystem = System(atomType)
end


0.0012005200551889156

In [3]:
function createSystem()
    BField(x, y, z) = 800E-4 * [0, 0, 1]
    atomType = cesiumAtom
    OurSystem = System(atomType, BField, -4e-6:0.4e-7:4e-6)
    return OurSystem
end
OurSystem = createSystem();

In [3]:
OurSystem.Environment

Environment_T(Any[], var"#BField#1"(), (-4.0e-6:4.0e-8:4.0e-6, -4.0e-6:4.0e-8:4.0e-6, -4.0e-6:4.0e-8:4.0e-6))

In [4]:
function createTweezer(OurSystem)
    clear_Tweezer(OurSystem)
    stateI_T = (6, 1, 1 / 2)
    stateF_T = (6, 0, 1 / 2)
    wavelength_tweezer = 1064E-9
    trapR, trapZ = 2 * pi * 0.150e6, 2 * pi * 0.029e6
    gridbounds = OurSystem.Environment.grid[1]
    spacing = gridbounds[2] - gridbounds[1]
    LocAll::Vector{Float64} = [0, 0, 0]
    println("Setting up Tweezer")
    #a = Tweezer(OurSystem.AtomType, BeamProperties(LocAll, [0.0, 0.0, 1.0], pol_v["pi"]), stateI_T, stateF_T, wavelength_tweezer, trapR, trapZ)
    #Tweezer_WF(a, OurSystem.Environment, spacing )
    set_tweezer(OurSystem, BeamProperties(LocAll, [0.0, 0.0, 1.0], pol_v["pi"]), stateI_T, stateF_T, wavelength_tweezer, trapR, trapZ)
end
createTweezer(OurSystem);

Setting up Tweezer
23854.61566735245323854.615667352453892.5625233054161


In [6]:
using BenchmarkTools
@btime OurSystem.TweezerConfig[1].ForceInterp(0, 0, 0)


  1.090 μs (8 allocations: 320 bytes)


3-element Vector{Float64}:
 3.916314366518297e-21
 3.916314366518297e-21
 1.4653581016695472e-22

In [4]:
stateI_T = (6, 1, 1 / 2)
stateF_T = (6, 0, 1 / 2)
wavelength_tweezer = 1064E-9
trapR, trapZ = 2 * pi * 0.150e6, 2 * pi * 0.029e6
LocAll::Vector{Float64} = [0, 0, 0]

set_tweezer(OurSystem, BeamProperties(LocAll, [0.0, 0.0, 1.0], pol_v["pi"]), stateI_T, stateF_T, wavelength_tweezer, trapR, trapZ)


23854.61566735245323854.615667352453892.5625233054161


1-element Vector{Any}:
 Tweezer_WF{ComplexF64}(942477.7960769379, 182212.373908208, GaussianBeam(BeamProperties{Vector{Float64}}([0.0, 0.0, 0.0], [0.0, 0.0, 1.0], ComplexF64[0.0 + 0.0im, 0.0 + 0.0im, 1.0 + 0.0im], 1.2387110694713286e-6, 4.5305223990725015e-6), 1.1451149418165695e7, 2.8176546804511275e14), 2.206946951453701e-25, Main.CoolTrap.var"#ForceInterp#26"{Tweezer{Float64}, Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}}(Tweezer{Float64}(942477.7960769379, 182212.373908208, GaussianBeam(BeamProperties{Vector{Float64}}([0.0, 0.0, 0.0], [0.0, 0.0, 1.0], ComplexF64[0.0 + 0.0im, 0.0 + 0.0im, 1.0 + 0.0im], 1.2387110694713286e-6, 4.5305223990725015e-6), 1.1451149418165695e7, 2.8176546804511275e14), 3.351227410563453e14, 1.1451149418165695e7, 2.206946951453701e-

In [ ]:
function createMOT(OurSystem)
    clear_MOT(OurSystem)
    stateF = (6, 1, 1.5, 5, 5)
    stateI = (6, 0, 0.5, 4, 4)
    detuning = 3e6
    I_mot = 14
    beamAngle = 45 * pi / 180


    println("Setting up MOT")
    updates0 = false
    LocAll::Vector{Float64} = [0, 0, 0]
    set_MOT(OurSystem, BeamProperties(LocAll, [-1.0, 0.0, 0.0], pol_v["sigma_m"]),  stateI, stateF, detuning, I_mot, updates0)
    set_MOT(OurSystem, BeamProperties(LocAll, [1.0, 0.0, 0.0], pol_v["sigma_p"]),  stateI, stateF, detuning, I_mot, updates0)
    #R1
    #set_MOT(OurSystem, BeamProperties(LocAll, [0, sin(beamAngle), cos(beamAngle)], pol_v["sigma_m"]),  stateI, stateF, detuning, I_mot,updates0)
    #set_MOT(OurSystem, BeamProperties(LocAll, [0, -sin(beamAngle), -cos(beamAngle)], pol_v["sigma_p"]),  stateI, stateF, detuning, I_mot, updates0)
    #R2
    #set_MOT(OurSystem, BeamProperties(LocAll, [0, -sin(beamAngle), cos(beamAngle)], pol_v["sigma_m"]),  stateI, stateF, detuning, I_mot, updates0)
    #set_MOT(OurSystem, BeamProperties(LocAll, [0, sin(beamAngle), -cos(beamAngle)], pol_v["sigma_p"]),  stateI, stateF, detuning, I_mot, updates0)
end

createMOT(OurSystem)

In [ ]:
clear_beams(OurSystem)
join_beams(OurSystem)
println("Number of Total Beams: ", length(OurSystem.BeamConfig))
println("Number of Tweezers Beams: ", length(OurSystem.TweezerConfig))
println("Number of MOT Beams: ", length(OurSystem.MOTConfig))

# Simulation 

In [ ]:
dt = 2e-9
tspan = (0, 4e-3)
sol = simulate(OurSystem, 1, 1e-3, "Fabs_spont_R", tspan, dt)

In [ ]:
timeEval, x, y, z, vx, vy, vz, photons = processData(sol)

In [ ]:
fig = plt.figure()
plot(timeEval*1e3, x[1]*1e6)
fig